# Download Music

In [2]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 19.5 MB/s eta 0:00:00


In [3]:
from yt_dlp import YoutubeDL

In [22]:
opts = {
    'format': 'bestaudio/best',
    'extract_audio': True,
    'audio_format': 'mp3',
    'audio_quality': 0,  # Best quality
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
    }],
    'postprocessor_args': [
        '-ss', '00:00:00',  # Start time (00:00:00)
        '-t', '00:00:45'    # Duration (45 seconds)
    ],
}

url = 'https://www.youtube.com/watch?v=kXYiU_JCYtU'

with YoutubeDL(opts) as ydl:
    ydl.download([url])

[youtube] Extracting URL: https://www.youtube.com/watch?v=kXYiU_JCYtU
[youtube] kXYiU_JCYtU: Downloading webpage
[youtube] kXYiU_JCYtU: Downloading ios player API JSON
[youtube] kXYiU_JCYtU: Downloading android player API JSON
[youtube] kXYiU_JCYtU: Downloading m3u8 information
[info] kXYiU_JCYtU: Downloading 1 format(s): 251
[download] Destination: Numb (Official Music Video) [4K UPGRADE] – Linkin Park [kXYiU_JCYtU].webm
[download] 100% of    2.76MiB in 00:00:00 at 24.94MiB/s  
[ExtractAudio] Destination: Numb (Official Music Video) [4K UPGRADE] – Linkin Park [kXYiU_JCYtU].mp3
Deleting original file Numb (Official Music Video) [4K UPGRADE] – Linkin Park [kXYiU_JCYtU].webm (pass -k to keep)


# Load Modules

In [5]:
import pandas as pd

import plotly.graph_objects as go

import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

from scipy.spatial.distance import cdist

import ast

# Load Model

## Load VA Generation

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, feature_dim):
        super(Attention, self).__init__()
        self.feature_dim = feature_dim
        self.attention = nn.Sequential(
            nn.Linear(feature_dim, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        scores = self.attention(x)
        alpha = F.softmax(scores, dim=1)
        attended_features = x * alpha
        return attended_features.view(-1, self.feature_dim)

class AudioNet(nn.Module):
    def __init__(self, params_dict):
        super(AudioNet, self).__init__()
        self.in_ch = params_dict.get('in_ch', 1)
        self.num_filters1 = params_dict.get('num_filters1', 32)
        self.num_filters2 = params_dict.get('num_filters2', 64)
        self.num_hidden = params_dict.get('num_hidden', 128)
        self.out_size = params_dict.get('out_size', 1)

        self.conv1 = nn.Sequential(
            nn.Conv1d(self.in_ch, self.num_filters1, kernel_size=10, stride=1),
            nn.BatchNorm1d(self.num_filters1),
            nn.ReLU(inplace=True),
            nn.AvgPool1d(kernel_size=2, stride=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(self.num_filters1, self.num_filters2, kernel_size=10, stride=1),
            nn.BatchNorm1d(self.num_filters2),
            nn.ReLU(inplace=True),
            nn.AvgPool1d(kernel_size=2, stride=2)
        )
        self.pool = nn.AvgPool1d(kernel_size=10, stride=10)

        self._to_linear = None
        self.attention = Attention(self._get_to_linear())

        self.fc1 = nn.Linear(self._get_to_linear(), self.num_hidden)
        self.fc2 = nn.Linear(self.num_hidden, self.out_size)
        self.drop = nn.Dropout(p=0.5)
        self.act = nn.ReLU(inplace=True)

    def _get_to_linear(self):
        if self._to_linear is None:
            x = torch.randn(1, self.in_ch, 4501)
            with torch.no_grad():
                x = self.conv1(x)
                x = self.conv2(x)
                x = self.pool(x)
                self._to_linear = x.numel() // x.shape[0]
        return self._to_linear

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = x.view(-1, self._get_to_linear())
        x = self.attention(x)
        x = self.fc1(x)
        x = self.drop(x)
        x = self.act(x)
        x = self.fc2(x)
        return x.to(x.device)

In [14]:
import librosa
import numpy as np

def extract_features(audio_path, sample_rate=44100):
    wave, sr = librosa.load(audio_path, sr=sample_rate)
    if len(wave) < sr * 45:
        wave = np.pad(wave, (0, sr * 45 - len(wave)), 'constant')
    wave = wave[:sr * 45]

    hop_length = int(sr * 0.01)
    win_length = int(sr * 0.025)

    mfcc = librosa.feature.mfcc(y=wave, sr=sr, n_mfcc=20, n_fft=2048, hop_length=hop_length, win_length=win_length)
    chroma = librosa.feature.chroma_stft(y=wave, sr=sr, n_fft=2048, hop_length=hop_length)
    contrast = librosa.feature.spectral_contrast(y=wave, sr=sr, n_fft=2048, hop_length=hop_length)

    return np.concatenate((mfcc, chroma, contrast), axis=0)


def predict(model, features):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    features = torch.tensor(features, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(features)
    return output.cpu().numpy()

In [15]:
class Predictor:
    def __init__(self, model_path_valence, model_path_arousal):
        self.model_path_valence = model_path_valence
        self.model_path_arousal = model_path_arousal
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        valence_params = {
            "in_ch": 39, "num_filters1": 32, "num_filters2": 64, "num_hidden": 64, "out_size": 1
        }
        arousal_params = {
            "in_ch": 39, "num_filters1": 32, "num_filters2": 32, "num_hidden": 128, "out_size": 1
        }

        self.valence_model = self.load_model(model_path_valence, valence_params)
        self.arousal_model = self.load_model(model_path_arousal, arousal_params)

    def load_model(self, model_path, params):
        model = AudioNet(params)
        model.load_state_dict(torch.load(model_path, map_location=self.device))
        model.to(self.device)
        model.eval()
        return model

    def extract_features(self, audio_path):
        sample_rate = 44100
        wave, sr = librosa.load(audio_path, sr=sample_rate)
        if len(wave) < sr * 45:
            wave = np.pad(wave, (0, sr * 45 - len(wave)), 'constant')
        wave = wave[:sr * 45]

        hop_length = int(sr * 0.01)
        win_length = int(sr * 0.025)

        mfcc = librosa.feature.mfcc(y=wave, sr=sr, n_mfcc=20, n_fft=2048, hop_length=hop_length, win_length=win_length)
        chroma = librosa.feature.chroma_stft(y=wave, sr=sr, n_fft=2048, hop_length=hop_length)
        contrast = librosa.feature.spectral_contrast(y=wave, sr=sr, n_fft=2048, hop_length=hop_length)

        features = np.concatenate((mfcc, chroma, contrast), axis=0)
        features_tensor = torch.tensor(features, dtype=torch.float32).unsqueeze(0)
        return features_tensor.to(self.device)

    def predict(self, audio_path):
        features = self.extract_features(audio_path)
        with torch.no_grad():
            valence_prediction = self.valence_model(features)
            arousal_prediction = self.arousal_model(features)
        return valence_prediction.item(), arousal_prediction.item()

In [16]:
import math

emotions = {
    "Sleepy": {"valence": 0.01, "arousal": -1.00},
    "Tired": {"valence": -0.01, "arousal": -1.00},
    "Afraid": {"valence": -0.12, "arousal": 0.79},
    "Angry":{"valence": -0.40, "arousal": 0.79},
    "Calm":{"valence": 0.78, "arousal": -0.68},
    "Relaxed":{"valence": 0.71, "arousal": -0.65},
    "Content":{"valence": 0.81, "arousal": -0.55},
    "Depressed":{"valence": -0.81, "arousal": -0.48},
    "Discontent":{"valence": -0.68, "arousal": -0.32},
    "Determined":{"valence": 0.73, "arousal": 0.26},
    "Happy":{"valence": 0.89, "arousal": 0.17},
    "Anxious":{"valence": -0.72, "arousal": -0.80},
    "Good":{"valence": 0.90, "arousal": -0.08},
    "Pensive":{"valence": 0.03, "arousal": -0.60},
    "Impressed":{"valence": 0.39, "arousal": -0.06},
    "Frustrated":{"valence": -0.60, "arousal": 0.40},
    "Disappointed":{"valence": -0.80, "arousal": -0.03},
    "Bored":{"valence": -0.35, "arousal": -0.78},
    "Annoyed":{"valence": -0.44, "arousal": 0.76},
    "Enraged":{"valence": -0.18, "arousal": 0.83},
    "Excited":{"valence": 0.70, "arousal": 0.71},
    "Melancholy":{"valence": -0.05, "arousal": -0.65},
    "Satisfied":{"valence": 0.77, "arousal": -0.63},
    "Distressed":{"valence": -0.71, "arousal": 0.55},
    "Uncomfortable":{"valence": -0.68, "arousal": -0.37},
    "Worried":{"valence": -0.07, "arousal": -0.32},
    "Amused":{"valence": 0.55, "arousal": 0.19},
    "Apathetic":{"valence": -0.20, "arousal": -0.12},
    "Peaceful":{"valence": 0.55, "arousal": -0.80},
    "Contemplative":{"valence": 0.58, "arousal": -0.60},
    "Embarrassed":{"valence": -0.31, "arousal": -0.60},
    "Sad":{"valence": -0.81, "arousal": -0.40},
    "Hopeful":{"valence": 0.61, "arousal": -0.30},
    "Pleased":{"valence": 0.89, "arousal": -0.10},
}

def find_emotion(valence, arousal):
    closest_emotion = None
    min_distance = math.inf

    for emotion, scores in emotions.items():
        distance = math.sqrt((valence - scores["valence"])**2 + (arousal - scores["arousal"])**2)

        if distance < min_distance:
            min_distance = distance
            closest_emotion = emotion

    return closest_emotion

clustered_emotions = {'blue': ['Determined',
  'Happy',
  'Good',
  'Impressed',
  'Excited',
  'Amused',
  'Hopeful',
  'Pleased'],
 'red': ['Depressed',
  'Discontent',
  'Anxious',
  'Disappointed',
  'Bored',
  'Uncomfortable',
  'Worried',
  'Apathetic',
  'Embarrassed',
  'Sad'],
 'green': ['Afraid', 'Angry', 'Frustrated', 'Annoyed', 'Enraged', 'Distressed'],
 'purple': ['Sleepy',
  'Tired',
  'Calm',
  'Relaxed',
  'Content',
  'Pensive',
  'Melancholy',
  'Satisfied',
  'Peaceful',
  'Contemplative']}


def get_colormap(valence, arousal):
    valence, arousal = normalize_value(valence), normalize_value(arousal)
    emotion = find_emotion(valence, arousal)
    for color, emotion_list in clustered_emotions.items():
        if emotion in emotion_list:
            return color
    return None

def normalize_value(value):
    return (value - 1) / 4 - 1

## Load Genre prediction

In [17]:
import torch
import torch.nn as nn

class MusicGenreClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MusicGenreClassifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.network(x)

model = MusicGenreClassifier(input_size=57, num_classes=10)
model.load_state_dict(torch.load('genre_classifier_model.pth'))
model.eval()

MusicGenreClassifier(
  (network): Sequential(
    (0): Linear(in_features=57, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=256, out_features=128, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=128, out_features=64, bias=True)
    (13): ReLU()
    (14): Dropout(p=0.3, inplace=False)
    (15): Linear(in_features=64, out_features=10, bias=True)
    (16): Softmax(dim=1)
  )
)

In [18]:
import librosa
import numpy as np

def extract_features(audio_path):
    y, sr = librosa.load(audio_path, sr=None)
    features = []
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    features.extend([np.mean(chroma_stft), np.var(chroma_stft)])
    rms = librosa.feature.rms(y=y)
    features.extend([np.mean(rms), np.var(rms)])
    spec_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    features.extend([np.mean(spec_centroid), np.var(spec_centroid)])
    spec_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    features.extend([np.mean(spec_bandwidth), np.var(spec_bandwidth)])
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    features.extend([np.mean(rolloff), np.var(rolloff)])
    zero_cross_rate = librosa.feature.zero_crossing_rate(y)
    features.extend([np.mean(zero_cross_rate), np.var(zero_cross_rate)])
    harmony = librosa.effects.harmonic(y)
    features.extend([np.mean(harmony), np.var(harmony)])
    percussive = librosa.effects.percussive(y)
    features.extend([np.mean(percussive), np.var(percussive)])
    tempo = librosa.beat.tempo(y=y, sr=sr, aggregate=None)
    features.append(np.mean(tempo))
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    for mfcc in mfccs:
        features.extend([np.mean(mfcc), np.var(mfcc)])

    return np.array(features)

# Load Dataset

In [34]:
df = pd.read_csv("spotify_va.csv")

## Extract Features from Music
Predictions include: valence, arousal, genre and colour

In [30]:
audio_path = 'Numb.mp3'
model_path_valence = 'model_valence.pth'
model_path_arousal = 'model_arousal.pth'

predictor = Predictor(model_path_valence, model_path_arousal)
valence, arousal = predictor.predict(audio_path)
print(f"Valence: {valence}, Arousal: {arousal}")
color = get_colormap(valence, arousal)
print(f'Emotion Detected: {color}')

features = extract_features(audio_path)
feature_values = torch.tensor(features, dtype=torch.float32).unsqueeze(0)

model.eval()
with torch.no_grad():
    outputs = model(feature_values)
    predicted_genre_index = outputs.argmax(dim=1).item()

genre_mapping = {0: 'blues', 1: 'classical', 2: 'country', 3: 'disco', 4: 'hiphop',
                 5: 'jazz', 6: 'metal', 7: 'pop', 8: 'reggae', 9: 'rock'}

genre = genre_mapping[predicted_genre_index]
print(f"The predicted genre of the song is: {genre}")

Valence: 4.940221309661865, Arousal: 5.504947662353516
Emotion Detected: red


<ipython-input-18-0bbf0ff63273>:23: FutureWarning:

librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.



The predicted genre of the song is: rock


Valence: 4.940221309661865, Arousal: 5.504947662353516
Emotion Detected: red
<ipython-input-18-0bbf0ff63273>:23: FutureWarning:

librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.

The predicted genre of the song is: rock

In [35]:
new_row = {
    'spotify_id': 'new_id',
    'artist': 'New Artist',
    'track': audio_path.split(".")[0],
    'file_path': audio_path,
    'genre': genre,
    'valence': valence,
    'arousal': arousal,
    'colour': color
}

# Add the new row to the DataFrame
df = df._append(new_row, ignore_index=True)

In [36]:
def filter_genre(df, genre):
  if genre == "blues" or genre == "jazz":
    filtered_df = df[df["genre"] == "blues"]
  elif genre == "raggae" or genre == "classical":
    filtered_df = df
  else:
    filtered_df = df[df["genre"] == genre]
  return filtered_df

filtered_df = filter_genre(df, genre)

# Build class

Might have to change the params for each shape when we settle on the spotify dataset

In [37]:
class MusicOnTrajectory:
    def __init__(self, df, shape):
        self.df = df
        self.shape = shape

    def run(self):
      closest_songs = self.shape.find_closest_songs(self.df)
      self.shape.plot_closest_points(closest_songs)

In [38]:
point = (valence, arousal)
point

(4.940221309661865, 5.504947662353516)

## Line

In [39]:
class Line:
    def __init__(self, point):
        self.x = point[0]
        self.y = point[1]
        self.m = None
        self.c = None

    def set_slope(self, m):
      self.m = m

    def set_intercept(self, c):
      self.c = c

    def equation(self, x):
        return self.m * x + self.c

    def perpendicular_distance(self, x, y):
        y_line = self.equation(x)
        return np.abs(y - y_line) / np.sqrt(self.m**2 + 1)

    def set_m_c(self, df):
      other_x = df.loc[:, 'valence'].values
      other_y = df.loc[:, 'arousal'].values
      # Calculate the slope of the line passing through the specific point
      # and minimize the error with the other points
      m = np.sum((other_x - self.x) * (other_y - self.y)) / np.sum((other_x - self.x)**2)
      c = self.y - m * self.x
      self.set_slope(m)
      self.set_intercept(c)

    def find_closest_songs(self, df):
        self.set_m_c(df)
        df['distance_to_line'] = self.perpendicular_distance(df['valence'], df['arousal'])
        closest_songs = df.sort_values(by='distance_to_line').head(10)
        return closest_songs

    def plot_closest_points(self, closest_songs):
        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x=closest_songs['valence'],
            y=closest_songs['arousal'],
            mode='markers',
            marker=dict(color=closest_songs['colour']),
            text=closest_songs['track'],
            hoverinfo='text+x+y',  # Display track and artist information on hover
            showlegend=False
        ))

        x_values = np.linspace(min(closest_songs['valence']), max(closest_songs['valence']), 100)
        y_values = self.equation(x_values)
        fig.add_trace(go.Scatter(
            x=x_values,
            y=y_values,
            mode='lines',
            line=dict(color='black'),
            name='Best-Fit Line'
        ))

        fig.update_layout(
            xaxis_title='Valence',
            yaxis_title='Arousal',
            title='Valence-Arousal Graph'
        )
        fig.show()

        print("--- Closest tracks to trajectory ---")
        for i, track in enumerate(closest_songs['track'], start=1):
            print(f"{i}: {track}")

In [40]:
l = Line(point)
t1 = MusicOnTrajectory(filtered_df, l)

t1.run()

<ipython-input-39-366363c010cd>:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



--- Closest tracks to trajectory ---
1: Numb
2: Capricorn
3: Vampire Empire
4: Pink Skies
5: Revival
6: Lil Tune
7: Vampire Disco
8: Mustang
9: Overcompensate (edit)
10: No Surprises


## Circle

In [41]:
class Circle:
    def __init__(self, point):
        self.x = point[0]
        self.y = point[1]
        self.x_circle = None
        self.y_circle = None

    def calculate_radius(self, df, scale = 0.5):
        radius = scale * np.sqrt((df['valence'] - self.x)**2 + (df['arousal'] - self.y)**2).max()
        return radius

    def set_circle_points(self, radius):
        theta = np.linspace(0, 2 * np.pi, 9)
        circle_points_x = self.x + radius * np.cos(theta)
        circle_points_y = self.y + radius * np.sin(theta)

        self.x_circle = circle_points_x
        self.y_circle = circle_points_y

        return circle_points_x, circle_points_y

    def find_closest_songs(self, df):
      radius = self.calculate_radius(df)
      circle_points_x, circle_points_y = self.set_circle_points(radius)
      points = []
      visited_indexes = set()
      for i in range(len(circle_points_x)):
          distances = np.sqrt((df['valence'] - circle_points_x[i])**2 + (df['arousal'] - circle_points_y[i])**2)
          closest_indexes = np.argsort(distances)
          for index in closest_indexes:
              if index not in visited_indexes and index != len(df)-1:
                  points.append(index)
                  visited_indexes.add(index)
                  break
      closest_songs = df.iloc[points]
      return closest_songs

    def plot_closest_points(self, closest_songs):
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=closest_songs['valence'],
            y=closest_songs['arousal'],
            mode='markers',
            marker=dict(color=closest_songs['colour']),
            text=closest_songs['track'],
            hoverinfo='text+x+y',
            showlegend=False
        ))

        input = df[(df['valence'] == self.x) & (df['arousal'] == self.y)]
        fig.add_trace(go.Scatter(
          x=[self.x],
          y=[self.y],
          mode='markers',
          marker=dict(color=input['colour']),
          text=input['track'],
          hoverinfo='text+x+y',
          showlegend=False
      ))


        fig.add_trace(go.Scatter(x=self.x_circle, y=self.y_circle, mode='lines', line=dict(color='black'), name='Circle'))

        fig.update_layout(
            xaxis_title='Valence',
            yaxis_title='Arousal',
            title='Valence-Arousal Graph')

        fig.show()

        print("--- Closest tracks to trajectory ---")
        print(f"{1}: {input['track'].values[0]}")
        for i, track in enumerate(closest_songs['track'], start=2):
            print(f"{i}: {track}")

In [42]:
c = Circle(point)
t2 = MusicOnTrajectory(filtered_df, c)

t2.run()

--- Closest tracks to trajectory ---
1: Numb
2: Burn The Witch
3: Planet
4: Good Old Days
5: Tejano Blue
6: Broken Man
7: Maktub
8: Thunder
9: Blondie
10: Me, Myself & Your Eyes


## Triangle

In [50]:
class Triangle:
    def __init__(self, point):
        self.x = point[0]
        self.y = point[1]
        self.points = self.generate_triangle_points()

    def generate_triangle_points(self, num_points=3, length=4):
      angles = np.linspace(0, 2*np.pi, num_points, endpoint=False)
      return (self.x, self.y) + np.column_stack((length*np.cos(angles), length*np.sin(angles)))

    def find_closest_songs(self, df):
        other_x = df['valence'].values
        other_y = df['arousal'].values

        other_points = np.column_stack((other_x, other_y))

        distances = cdist(other_points, self.points)
        nearest_indexes = np.argsort(distances, axis=0)[:11].flatten()

        nearest_indexes = [d for d in nearest_indexes if not np.array_equal(d, (self.x, self.y))]
        random_indexes = np.random.choice(nearest_indexes, size=9, replace=False)
        closest_songs = df.iloc[random_indexes]
        return closest_songs

    def plot_closest_points(self, closest_songs):
        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x=closest_songs['valence'],
            y=closest_songs['arousal'],
            mode='markers',
            marker=dict(color=closest_songs['colour']),
            text=closest_songs['track'],
            hoverinfo='text+x+y',
            showlegend=False
        ))

        input = df[(df['valence'] == self.x) & (df['arousal'] == self.y)]
        fig.add_trace(go.Scatter(
          x=[self.x],
          y=[self.y],
          mode='markers',
          marker=dict(color=input['colour']),
          text=input['track'],
          hoverinfo='text+x+y',
          showlegend=False
      ))

        fig.add_trace(go.Scatter(
          x=np.append(self.points[:, 0], self.points[0, 0]),
          y=np.append(self.points[:, 1], self.points[0, 1]),
          mode='lines',
          marker=dict(color='black'),
          name='Triangle')
        )

        fig.update_layout(
            xaxis_title='Valence',
            yaxis_title='Arousal',
            title='Valence-Arousal Graph')

        fig.show()

        print("--- Closest tracks to trajectory ---")
        print(f"{1}: {input['track'].values[0]}")
        for i, track in enumerate(closest_songs['track'], start=2):
            print(f"{i}: {track}")

In [51]:
t = Triangle(point)
t3 = MusicOnTrajectory(filtered_df, t)

t3.run()

--- Closest tracks to trajectory ---
1: Numb
2: Northstar
3: Dead To Me
4: Me, Myself & Your Eyes
5: You Know What You’ve Done
6: Good Old Days
7: I Forgot To Be Your Lover
8: Tiny Moves
9: FACETIME
10: DOWNSIDE


## Parabola

In [56]:
class Parabola:
    def __init__(self, point):
        self.x = point[0]
        self.y = point[1]
        self.points = self.parabola_points()

    def parabola_points(self, num_points=10, a=1, b=0, c=0, shift=5, scale=0.4):
        x = np.linspace(-5 + shift, 5 + shift, num_points)
        y = (a * (x - shift)**2 + b * (x - shift) + c) * scale
        return np.column_stack((x, y))

    def find_closest_songs(self, df):
        closest_indices = []
        visited_indices = set()

        for i in range(len(self.points)):
            distances = np.sqrt((df['valence'] - self.points[i][0])**2 + (df['arousal'] - self.points[i][1])**2)
            sorted_indices = np.argsort(distances)

            for index in sorted_indices:
                if index not in visited_indices:
                    closest_indices.append(index)
                    visited_indices.add(index)
                    break

        closest_songs = df.iloc[closest_indices]
        return closest_songs

    def plot_closest_points(self, closest_songs):
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=closest_songs['valence'],
            y=closest_songs['arousal'],
            mode='markers',
            marker=dict(color=closest_songs['colour']),
            text=closest_songs['track'],
            hoverinfo='text+x+y',
            showlegend=False
        ))

        input_point = df[(df['valence'] == self.x) & (df['arousal'] == self.y)]
        fig.add_trace(go.Scatter(
            x=[self.x],
            y=[self.y],
            mode='markers',
            marker=dict(color=input_point['colour']),
            text=input_point['track'],
            hoverinfo='text+x+y',
            showlegend=False
        ))

        fig.add_trace(go.Scatter(x=self.points[:, 0], y=self.points[:, 1], mode='lines', line=dict(color='black'), name='Parabola'))

        fig.update_layout(
            xaxis_title='Valence',
            yaxis_title='Arousal',
            title='Valence-Arousal Graph')

        fig.show()

        print("--- Closest tracks to trajectory ---")
        print(f"{1}: {input_point['track'].values[0]}")
        for i, track in enumerate(closest_songs['track'], start=2):
            print(f"{i}: {track}")

In [57]:
p = Parabola(point)
t4 = MusicOnTrajectory(filtered_df, p)

t4.run()

--- Closest tracks to trajectory ---
1: Numb
2: Tejano Blue
3: Broken Man
4: Tongue Tied
5: Too Much
6: Heart To Heart
7: You Know What You’ve Done
8: Believer
9: Blondie
10: DOWNSIDE
11: kinda smacks
